In [9]:
import tensorflow as tf
import numpy as np

import pandas as pd
import cv2
import os
import math
import scipy as sp
import PIL

# Tensorflow
from tensorflow.keras import models, layers, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Flatten, Dense, Dropout, ZeroPadding2D

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import EfficientNetB4, EfficientNetB6, ResNet50V2
#from keras_tuner.tuners import RandomSearch

import scikitplot as skplt
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [2]:
import glob
import pandas as pd

filenames = []
Pokemonname = []
# root_dir needs a trailing slash (i.e. /root/dir/)
for filename in glob.iglob("dataset/" + '**/*.jpg', recursive=True):
    Pokemonname.append(filename.split('/')[1])
    filenames.append(filename)

In [3]:
filenames[0].split('/')

['dataset', 'Zapdos', 'c3f34230c7e24d18845c4d2eaad54801.jpg']

In [4]:
df = pd.read_csv('./FirstGenPokemon.csv')
df.head(10)

,PKN,Name,Type1
0,1,Bulbasaur,grass
1,2,Ivysaur,grass
2,3,Venusaur,grass
3,4,Charmander,fire
4,5,Charmeleon,fire
5,6,Charizard,fire
6,7,Squirtle,water
7,8,Wartortle,water
8,9,Blastoise,water
9,10,Caterpie,grass


In [5]:
ImageData = pd.DataFrame({
    'FilePath': filenames,
    'Name': Pokemonname
})

ImageData.head()


,FilePath,Name
0,dataset/Zapdos/c3f34230c7e24d18845c4d2eaad5480...,Zapdos
1,dataset/Zapdos/f5a52cd93f924e28a46d7def3265d6b...,Zapdos
2,dataset/Zapdos/26b19f8809ce496eae2e1b822d54492...,Zapdos
3,dataset/Zapdos/7a63f55b762a4fe7b6cd92f17ea22eb...,Zapdos
4,dataset/Zapdos/0b9b5e421d8949e0b2b6698c8d0bb92...,Zapdos


In [6]:
df['Type1'].value_counts()

water       32
normal      31
grass       24
rock        17
poison      14
fire        12
psychic     11
electric    10
Name: Type1, dtype: int64

In [8]:
DATA = pd.merge(ImageData, df, on='Name')

DATA.head()

,FilePath,Name,PKN,Type1
0,dataset/Zapdos/c3f34230c7e24d18845c4d2eaad5480...,Zapdos,145,electric
1,dataset/Zapdos/f5a52cd93f924e28a46d7def3265d6b...,Zapdos,145,electric
2,dataset/Zapdos/26b19f8809ce496eae2e1b822d54492...,Zapdos,145,electric
3,dataset/Zapdos/7a63f55b762a4fe7b6cd92f17ea22eb...,Zapdos,145,electric
4,dataset/Zapdos/0b9b5e421d8949e0b2b6698c8d0bb92...,Zapdos,145,electric


In [10]:
# batch_size: 한번에 forward & Backword 하는 샘플의 수
batch_size = 16

# Training 수
epochs = 50

# Weight 조절 parameter
LearningRate = 1e-3 # 0.001
Decay = 1e-6

img_width = 112
img_height = 112

In [11]:
# 모델 Return
VGGModel = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_tensor=None, input_shape=(img_width,img_height,3), pooling=None)
x = GlobalAveragePooling2D()(VGGModel.output)
predictions = Dense(8, activation='softmax')(x)

DeepLearning = Model(inputs=VGGModel.input, outputs=predictions)

# learning parameter를 더하여 최종 model compile
DeepLearning.compile(optimizer=
         SGD(lr=LearningRate, decay=Decay, momentum=0.9, nesterov=True), 
         loss='categorical_crossentropy',
         metrics=['acc']
) # 나이를, MSE


94781440/94765736 [==============================] - 5s 0us/step


/Users/krc/miniforge3/envs/dl/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [12]:
# Online-augmentation 적용 Generator
# 1. 이미지를 전부다 불러서 램 (메모리)에 올릴 수 없기 때문
# 2. 이미지는 Augmentation을 해주는게 좋아서

DATAGEN_TRAIN = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last",
    validation_split=0.10) # Train / Validation

# Online-augmentation 비적용 Generator (Test용)
DATAGEN_TEST = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    data_format="channels_last")

In [13]:
train_generator=DATAGEN_TRAIN.flow_from_dataframe(
dataframe=DATA,
directory="./",
x_col="FilePath",
y_col="Type1",
subset="training",
batch_size=16,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 9107 validated image filenames belonging to 8 classes.


In [14]:
valid_generator=DATAGEN_TRAIN.flow_from_dataframe(
dataframe=DATA,
directory="./",
x_col="FilePath",
y_col="Type1",
subset="validation",
batch_size=16,
seed=42,
shuffle=True,
class_mode="categorical",
target_size=(224,224))

Found 1011 validated image filenames belonging to 8 classes.


In [ ]:
test_generator = DATAGEN_TEST.flow_from_dataframe(
    dataframe=DATA,
    directory="./",
    x_col="FilePath",
    y_col="Type1",
    batch_size=16,
    seed=42,
    shuffle=False,
    class_mode="categorical",
    target_size=(224,224)
)

In [15]:
########## Training Start
DeepLearning.fit(
        train_generator,
        # 데이터가 너무 클 경우 1-epoch을 못하는 경우
        # steps_per_epoch=TRAIN_GENERATOR.n / batch_size,
        
        epochs=15,
        #callbacks=CALLBACK,
        shuffle=True, # Training에 패턴이 존재하면 overfit이 잘 되기 때문에, Shuffle 사용해야함. 단 test에는 절대 X
        validation_data=valid_generator)
###########

/Users/krc/miniforge3/envs/dl/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/Users/krc/miniforge3/envs/dl/lib/python3.8/site-packages/keras_preprocessing/image/image_data_generator.py:728: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


Epoch 1/15


2022-07-29 14:23:20.697373: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


285/285 [==============================] - 1266s 4s/step - loss: 1.1944 - acc: 0.5812 - val_loss: 2.6231 - val_acc: 0.1167
Epoch 2/15
285/285 [==============================] - 1295s 5s/step - loss: 0.6495 - acc: 0.7794 - val_loss: 3.3403 - val_acc: 0.1860
Epoch 3/15
285/285 [==============================] - 1260s 4s/step - loss: 0.4603 - acc: 0.8446 - val_loss: 2.9298 - val_acc: 0.3422
Epoch 4/15
285/285 [==============================] - 1234s 4s/step - loss: 0.3534 - acc: 0.8806 - val_loss: 1.8308 - val_acc: 0.5015
Epoch 5/15
285/285 [==============================] - 1232s 4s/step - loss: 0.2776 - acc: 0.9068 - val_loss: 1.8451 - val_acc: 0.4965
Epoch 6/15
118/285 [===========>..................] - ETA: 11:36 - loss: 0.2190 - acc: 0.9280

KeyboardInterrupt: 

In [19]:
DeepLearning.save('iris.h5')